In Quick Start, we learned the basics of HyperTS modeling:

In [ ]:
from hyperts.experiment import make_experiment
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

df = load_network_traffic()
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

experiment = make_experiment(train_data,
                            task='forecast',
                            timestamp='TimeStamp',
                            covariables=['HourSin', 'WeekCos', 'CBWD'])
model = experiment.run()

X_test, y_test = model.split_X_y(test_data)
forecast = model.predict(X_test)
scores = model.evaluate(y_true=y_test, y_pred=forecast)

To get a better understanding of HyperTS, this NoteBook will go through the ```make_experiment``` tutorial in more detail, so you can explore more robust performance.

1. Create a experiment by default.
2. Select operation mode.
3. Specify the evaluation metrics.
4. Specify optimization direction.
5. Set the maximum search trials.
6. Set early stoping.
7. Specify the validation data set.
8. Specify search algorithm.
9. Specify time frequency.
10. Specify forecast window.
11. Fixed random seed.
12. Adjusting a log level.

#### 1. Create a experiment by default.

In [ ]:
# Forecast
experiment = make_experiment(train_data, 
                             task='forecast',
                             timestamp='TimeStamp',
                             covariables=['HourSin', 'WeekCos', 'CBWD'])
# Classification
experiment = make_experiment(train_data, task='classification', target='y')  

# Regression
experiment = make_experiment(train_data, task='regression', target='y')  

#### 2. Select operation mode.

In [ ]:
experiment = make_experiment(train_data, 
                             mode='dl',
                             ...)

In [ ]:
experiment = make_experiment(train_data, 
                             mode='dl',
                             dl_gpu_usage_strategy=1,
                             ...)  

#### 3. Specify the evaluation metrics.

In [ ]:
# str
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric='auc',
                             ...)  

# sklearn.metrics
from sklearn.metrics import auc
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric=auc,
                             ...) 

#### 4. Specify optimization direction.

In [ ]:
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric='auc',
                             optimize_direction='max',
                             ...)  

#### 5. Set the maximum search trials.

In [ ]:
experiment = make_experiment(train_data, 
                             max_trials=100,
                             ...) 

#### 6. Set early stoping.

In [ ]:
experiment = make_experiment(train_data, 
                             max_trials=100,
                             early_stopping_time_limit=3600 * 3,  # 3 hours
                             ...)    

#### 7. Specify the validation data set.

In [ ]:
experiment = make_experiment(train_data, 
                             eval_data=eval_data,
                             ...) 

In [ ]:
experiment = make_experiment(train_data, 
                             eval_size=0.3,
                             ...) 

#### 8. Specify search algorithm.

In [ ]:
experiment = make_experiment(train_data, 
                             searcher='random',
                             ...)  

In [ ]:
from hypernets.searchers import EvolutionSearcher

search_space_general = ...

experiment = make_experiment(train_data, 
                             searcher=EvolutionSearcher(search_space_general, population_size=500, sample_size=20, candidates_size=20),
                             ...)  

#### 9. Specify time frequency.

In [ ]:
experiment = make_experiment(train_data, 
                             task='forecast',
                             timestamp='TimeStamp',
                             freq='H',
                             ...) 

#### 10. Specify forecast window.

In [ ]:
experiment = make_experiment(train_data, 
                             task='forecast',
                             mode='dl',
                             timestamp='TimeStamp',
                             forecast_window=24*7,
                             ...)

#### 11. Fixed random seed.

In [ ]:
experiment = make_experiment(train_data, 
                             random_state=0,
                             ...)  

#### 12. Adjusting a log level.

In [ ]:
experiment = make_experiment(train_data, 
                             log_level='INFO', 
                             verbose=1,
                             ...)  